
# Исследование месторождений нефти 

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных

**Признаки**
- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);

**Целевой признак**
- `product` — объём запасов в скважине (тыс. баррелей).

### Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

### Содержание:

[1. Загрузка и подготовка данных](#step1) <br>
[2. Обучение и проверка модели](#step2) <br>
[3. Подготовка к расчету прибыли](#step3)<br>
[4. Расчет прибыли и рисков](#step4)<br>
[5. Общий вывод](#step5)<br>

<a id='step1'></a>
# 1. Загрузка и подготовка данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
pd.options.mode.chained_assignment = None

In [2]:
#открываем файлы с данными
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [7]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Во всех трех датасетах отсутствуют пропущенные значения, аномальные значения также не выявлены. Признаки по условию значимы, но мы не знаем, что они означают, поэтому изменять их или удалять опасно.

In [9]:
#удаляем из датасетов ненужный столбец id
datasets = [data_0, data_1, data_2]
for data in datasets:
    data.drop('id', axis=1, inplace=True)

### Вывод

Мы получили три датасета без пропущенных и аномальных значений, удалили не несущий полезной информации столбец `id` и теперь можем приступать к обучению моделей.

<a id='step2'></a>
# 2. Обучение и проверка модели

In [10]:
%%time
state = np.random.RandomState(12345)
sets = len(datasets)

#в цикле разделим датасеты и обучим модели линейной регрессии
for i in range(sets):
    X = datasets[i].drop('product', axis=1)
    y = datasets[i]['product']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=i)
    
    #сохраним правильные ответы в переменных y_i_valid, где i - номер датасета
    locals()['y_' + str(i) + '_valid'] = y_valid

    model = LinearRegression().fit(X_train, y_train)

    y_pred = model.predict(X_valid)
    
    #сохраним предсказания модели в переменных y_i_pred, где i - номер датасета
    locals()['y_' + str(i) + '_pred'] = pd.Series(y_pred, index=y_valid.index)
    print(f'Модель для датасета №{i}')
    print(f'Средний запас предсказанного сырья: {y_pred.mean():.2f}')
    print(f'RMSE: {mse(y_valid, y_pred)**0.5:.2f}')
    print('-----')

Модель для датасета №0
Средний запас предсказанного сырья: 92.27
RMSE: 37.48
-----
Модель для датасета №1
Средний запас предсказанного сырья: 69.12
RMSE: 0.89
-----
Модель для датасета №2
Средний запас предсказанного сырья: 94.86
RMSE: 39.98
-----
CPU times: user 254 ms, sys: 205 ms, total: 459 ms
Wall time: 455 ms


### Вывод

Мы разделили три данных датафрейма на тестовые и валидационные выборки, обучили модель линейной регрессии и предсказали запас сырья для валидационных выборок. По итогу машинного обучения в регионе **№0** и **№2** средний запас предсказанного сырья равен **~92 и ~95** тыс. баррелей в скважине, тогда как для региона **№1** этот же показатель равен всего лишь **~69** тыс. баррелей, что заметно ниже. Однако рассчитанная метрика *RMSE* для регионов **№0** и **№2** показывает, что модель в среднем ошибается на **37** и **40** тыс. баррелей соответственно, что является показателем очень низкой точности нашей модели, тогда как для региона **№1** *RMSE* составляет меньше **1** тыс. баррелей, что говорит о высокой корреляции одного из (или всех) признаков к объему скважины, а также о том, что модель отлично обучилась. 

Задачей установлено, что мы можем использовать только модель линейной регрессии, поэтому придется работать с тем, что мы получили. Приступим к расчету прибыли.

<div class="alert alert-block alert-success">
<b>Успех:</b> Отличный вывод, основный факты отмечены!
</div>

<a id='step3'></a>
# 3. Подготовка к расчёту прибыли

In [11]:
#объявим переменные для будущих расчетов
#количество исследуемых скважин
explored_bore_holes = 500
#количество лучших скважин
best_bore_holes = 200
#бюджет на разработку
budget = 1e10
#стоимость одной тысячи баррелей (одной единице в наших данных)
barrel_unit_cost = 45e4

In [12]:
#рассчитаем необходимый объём сырья в новой скважине для безубыточной разработки
raw_volume_needed = budget / barrel_unit_cost / best_bore_holes
print(f'{raw_volume_needed:.2f} тыс. баррелей необходимо для безубыточной разработки новой скважины.')

111.11 тыс. баррелей необходимо для безубыточной разработки новой скважины.


### Вывод

Сравнивая полученное число объёма сырья в скважине, которое нам необходимо для выхода на прибыль, с полученными объемами предсказанного сырья в регионах, можно резюмировать, что основная масса новых скважин не доходит до необходимого нам уровня. Особенно сильно отстает регион **№2**, в котором в среднем практически в два раза меньше сырья в новой скважине. Однако, нам стоит исследовать только лучшие скважины с лучшими показателями, возможно, мы сможем преодолеть планку объёма, которую мы рассчитали.

<a id='step4'></a>
# 4. Расчёт прибыли и рисков 

In [13]:
#объявляем функцию для расчета прибыли
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return barrel_unit_cost * selected.sum() - budget

In [14]:
#объявляем функцию для применения техники Bootstrap и нахождения необходимых нам показателей
def income(target, probabilities):
    values = []
    for i in range(1000):

        target_subsample = target.sample(n=explored_bore_holes, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        rev = revenue(target_subsample, probs_subsample, best_bore_holes)
        values.append(pd.Series(rev).quantile(.99))
    
    values = pd.Series(values)
    
    #рассчитываем границы 95%-ного доверительного интервала 
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    #рассчитываем процент убыточных скважин
    bad_bore_holes = (values[values<0].count()/values.count())
    
    print(f'Средняя прибыль: {values.mean():.2f}')
    print(f'95%-доверительный интервал: {lower:.2f}, {upper:.2f}')
    print(f'Процент убыточных скважин: {bad_bore_holes:.2%}')
    print('---------------------------------')

In [15]:
#вызываем функции для трех регионов
print('Регион №0')
income(y_0_valid, y_0_pred)

print('Регион №1')
income(y_1_valid, y_1_pred)

print('Регион №2')
income(y_2_valid, y_2_pred)

Регион №0
Средняя прибыль: 472012951.48
95%-доверительный интервал: -60816457.82, 981151924.74
Процент убыточных скважин: 4.70%
---------------------------------
Регион №1
Средняя прибыль: 533056709.60
95%-доверительный интервал: 99430448.40, 973064424.89
Процент убыточных скважин: 0.60%
---------------------------------
Регион №2
Средняя прибыль: 329790492.06
95%-доверительный интервал: -227100329.60, 868590498.70
Процент убыточных скважин: 12.00%
---------------------------------


<a id='step5'></a>
# 5. Общий вывод

После применения техники Bootstrap и расчета средней прибыли и доверительного интервала мы получили такие данные:

**- наибольшая средняя прибыль в регионе №1**

**- есть определенная статистическая вероятность того, что разработка в регионах №0 и №2 принесет убыток** (нижняя граница доверительного интервала отрицательная)

**- процент убыточных скважин менее 2.5% только у региона №1**

Очевидным решением будет **предложить регион №1 как наиболее перспективный** для разработки новых скважин по имеющимся данным. Так же модель, обученная на данных этого региона, является лучшей для будущего прогнозирования объёма сырья.

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован